# LangChain Memory

In [ ]:
from utils.helpers import get_secret
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.embeddings import CohereEmbeddings
from langchain.chat_models import BedrockChat

In [ ]:
COHERE_KEY = get_secret("llm/api_keys/cohere")
PERSIST_DIR = "../docs/chroma/"

In [ ]:
embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=COHERE_KEY["cohere_api_key"])
llm = BedrockChat(model_id="anthropic.claude-v2", model_kwargs={"temperature": 0.4})


## Regular Memory

In [ ]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

## Memory with a specific window

In [ ]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

## Memory buffer keeping a summary

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=400)
memory.save_context({"Human": "Hello"}, {"Assistant": "What's up"})
memory.save_context({"Human": "Not much, just hanging"},
                    {"Assistant": "Cool"})
memory.save_context({"Human": "What is on the schedule today?"}, 
                    {"Assistant": f"{schedule}"})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
conversation.predict(input="I want to have dinner with my wife in the evening. What would be a safe time to reserve a table?")
